In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
def predict(messages, model, tokenizer):
    if torch.backends.mps.is_available():
        device = "mps"
    elif torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    # 关闭 gradient checkpointing 以避免 generate 时冲突
    model.gradient_checkpointing_disable()

    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids, 
        max_new_tokens=2048
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # 生成完后，重新启用 gradient checkpointing（方便后续再训练或别的操作）
    model.gradient_checkpointing_enable()
    return response

In [ ]:
# 加载原下载路径的tokenizer和model
tokenizer = AutoTokenizer.from_pretrained(
    "./output/Qwen3-1.7B/checkpoint-1084", 
    use_fast=False,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "./output/Qwen3-1.7B/checkpoint-1084", 
    device_map="auto", 
    dtype=torch.bfloat16
)


`torch_dtype` is deprecated! Use `dtype` instead!


In [8]:
test_texts = {
    'instruction': "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。",
    'input': "医生，我最近被诊断为糖尿病，听说碳水化合物的选择很重要，我应该选择什么样的碳水化合物呢？"
}

instruction = test_texts['instruction']
input_value = test_texts['input']

messages = [
    {"role": "system", "content": f"{instruction}"},
    {"role": "user", "content": f"{input_value}"}
]

response = predict(messages, model, tokenizer)
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<think>嗯，用户问的是关于糖尿病患者在饮食中应该选择哪些类型的碳水化合物。首先，我需要回忆一下糖尿病患者的饮食指南。记得糖尿病患者的饮食控制非常重要，尤其是碳水化合物的选择，因为碳水化合物直接影响血糖水平。

首先，应该区分复杂碳水化合物和简单碳水化合物。简单碳水化合物比如精制的糖类，比如白米饭、白面包，这些升糖较快，可能对血糖影响大，所以应该避免或者减少摄入。而复杂碳水化合物，比如全谷物、杂粮，或者富含膳食纤维的，比如燕麦、荞麦，这些升糖较慢，对血糖的影响较小，所以应该优先选择。

然后，用户提到的杂粮，比如玉米、绿豆、荞麦、燕麦，这些可能属于复杂碳水化合物，但需要确认它们的升糖指数是否较低。另外，膳食纤维含量高的食物，比如蔬菜、豆类，虽然主要不是碳水化合物，但膳食纤维可能帮助减缓碳水化合物的吸收，所以可能被推荐。

另外，答案中提到要选择富含膳食纤维的杂粮，这可能是指虽然这些杂粮本身是碳水化合物，但它们的纤维含量高，所以整体对血糖的影响较小。需要确认膳食纤维对血糖的影响，比如高纤维食物可能减缓碳水化合物的消化吸收，从而降低血糖波动。

还要考虑用户作为刚诊断糖尿病的患者，可能需要更具体的建议，比如哪些食物属于复杂碳水化合物，或者如何搭配。可能需要建议选择复合碳水化合物作为主食，比如全麦面包、糙米等，而精制碳水化合物作为偶尔的补充。

另外，可能要考虑血糖生成指数（GI）的概念，高GI的食物会导致血糖迅速上升，而低GI的食物则相反。复杂碳水化合物通常GI较低，所以应该选择这类。例如，燕麦的GI较低，而玉米的GI较高，所以可能需要避免。

还有，用户可能不太清楚膳食纤维的具体来源，是否需要建议增加蔬菜和豆类的摄入，因为它们虽然主要提供蛋白质和维生素，但膳食纤维含量高，同时作为碳水化合物的来源可能被推荐。不过可能需要明确说明这些属于复杂碳水化合物还是其他类型，但根据答案中的描述，可能被归类为复杂碳水化合物的一部分。

总结下来，思考过程应该是先确定复杂碳水化合物和简单碳水化合物的区别，然后列举具体的例子，强调低GI和高纤维的食物，最后给出建议。同时，考虑到患者可能需要具体的食谱或食物例子，但答案已经给出了一些选项，可能需要进一步解释为什么这些食物合适，以及如何搭配其他营养素。
</think> 
 您好，对于糖尿病患者来说，选择低GI值的复合碳水化合物非常重要。